In [1]:
from keras.preprocessing.sequence import TimeseriesGenerator
from FC_RNN_Evaluater.FC_RNN_Evaluater import *
from keras.initializers import RandomNormal
import numpy as np

/home/mcicek/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
timesteps = 10
inputMatrix = np.random.rand(57,224,224,3)# np.array([[[[i, i, i]]] for i in range(57)])
labels = np.array([[i, i, i] for i in range(57)])

inputMatrix, inputLabels, outputLabels = getSequencesToSequences(inputMatrix, labels, timesteps)
batch_size=1    
img_gen = TimeseriesGenerator(inputMatrix, outputLabels, length=timesteps, sampling_rate=1, stride=timesteps, batch_size=batch_size)
ang_gen = TimeseriesGenerator(inputLabels, outputLabels, length=timesteps, sampling_rate=1, stride=timesteps, batch_size=batch_size)
batch_01 = img_gen[0]
batch_0 = ang_gen[0]
inputMatrix, y = batch_01
inputLabels, outputLabels = batch_0

In [7]:
from FC_RNN_Evaluater.Stateful_FC_RNN_Configuration import *
from FC_RNN_Evaluater.FC_RNN_Evaluater import *
from FC_RNN_Evaluater.ReinforceAlgorithmForKerasModels import *

In [5]:
vgg_model, full_model, modelID, preprocess_input = getFinalModel(timesteps = timesteps, lstm_nodes = lstm_nodes, lstm_dropout = lstm_dropout, lstm_recurrent_dropout = lstm_recurrent_dropout, 
                      num_outputs = num_outputs, lr = learning_rate, include_vgg_top = include_vgg_top, use_vgg16 = use_vgg16)

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as k
from keras import losses
import numpy as np
import tensorflow as tf

In [14]:
trainingBiwi = readBIWIDataset(subjectList = trainingSubjects, preprocess_input = preprocess_input)

All frames and annotations from 1 datasets have been read by 2019-02-20 01:22:41.366291


In [25]:
def getGradientsPerEpisode(model, samples, targets):
    gradients_per_episode = []
    rewards = getRewardsWithBaselinePerEpisode(samples, targets)
    for i in range(samples.shape[0]):
        loss = losses.mean_squared_error(targets, samples[i])
        gradients = k.gradients(loss, model.trainable_weights)
        rewardedGradients = [g*rewards[i] for g in gradients]
        gradients_per_episode.append(rewardedGradients)
    return gradients_per_episode, tf.reduce_mean(rewards, axis=0).eval()

def getFinalGradients(model, samples, targets):
    gradients_per_episode, reward = getGradientsPerEpisode(model, samples, targets)
    stacked_gradients = []
    for i in range(len(gradients_per_episode[0])):
        stacked_gradients.append(tf.stack([gradients[i] for gradients in gradients_per_episode])) 
    return [tf.reduce_mean(g, axis=0) for g in stacked_gradients], reward


In [26]:
def reinforceModelForEpoch(model, data_gen, episodes, sigma, steps_per_epoch, epochCount, verbose=1):
    i = 0
    if verbose == 1: printProgressBar(0, steps_per_epoch, prefix = epochCount, suffix = 'Complete', length = 50)
    for (inputMatrix_batch, inputLabels_batch), outputLabels_batch in data_gen:
        tracker_outputs = model.predict([inputMatrix_batch, inputLabels_batch])
        samples = drawSamples(model, episodes, sigma, tracker_outputs)
        final_gradients, reward = getFinalGradients(model, samples, outputLabels_batch)
        model = getUpdatedModelWithGradients(model, final_gradients)
        i += 1
        if verbose == 1: printProgressBar(i, steps_per_epoch, prefix = epochCount, suffix = 'Complete, reward: %f' % reward, length = 50)
    return model

In [27]:
def reinforceModel(model, data_gen, episodes, sigma, steps_per_epoch, epochs, verbose=1):
    for e in range(epochs):
        epochCount = 'Epoch %d/%d:' % (e+1, epochs)
        model = reinforceModelForEpoch(model, data_gen, episodes, sigma, steps_per_epoch, epochCount, verbose)
    return model


In [28]:
set_gen = readBIWIDataset(subjectList = trainingSubjects, preprocess_input = preprocess_input)
for inputMatrix, labels in set_gen:
        labels = labels[:, output_begin:output_begin+num_outputs]
        inputMatrix, inputLabels, outputLabels = getSequencesToSequences(inputMatrix, labels, timesteps) 
        data_gen = combined_generator2(inputMatrix, inputLabels, outputLabels, timesteps, batch_size)
        reinforceModel(full_model, data_gen, 5, 0.02, steps_per_epoch, 1, verbose=1)

All frames and annotations from 1 datasets have been read by 2019-02-20 01:37:49.054218


ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`